# Steps to EQ-Rotta

In [1]:
# import necessities
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr

# from pdfs import *
from netCDF4 import Dataset
import os

In [2]:
def list_directories(path):
    try:
        # List all items in the given path
        items = os.listdir(path)
        print (path, items)
        # Filter out the directories
        
        directories = [item for item in items if os.path.isdir(os.path.join(path, item)) and item.startswith('Ug')]
        items = [item for item in items if item.startswith('Ug')]
        
        return  directories , items
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [3]:
# list directories using above function
searchpath = '/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2'
filenames = list_directories(searchpath)
filenames

/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2 ['Ug16Q006_I.nc', 'Ug8Q006_IV.nc', 'Ug8Q003_IV.nc', 'Ug16Q010_IV.nc', 'Ug2Q024_I.nc', 'Ug8Q003_III.nc', 'Ug16Q001_IV.nc', 'Ug2Q01_III.nc', 'Ug16Q003_IV.nc', 'Ug2Q010_I.nc', 'Ug1Q01_II.nc', 'Ug8Q003_II.nc', 'Ug16Q000_IV.nc', 'Ug2Q010_IV.nc', 'Ug16Q006_IV.nc']


([],
 ['Ug16Q006_I.nc',
  'Ug8Q006_IV.nc',
  'Ug8Q003_IV.nc',
  'Ug16Q010_IV.nc',
  'Ug2Q024_I.nc',
  'Ug8Q003_III.nc',
  'Ug16Q001_IV.nc',
  'Ug2Q01_III.nc',
  'Ug16Q003_IV.nc',
  'Ug2Q010_I.nc',
  'Ug1Q01_II.nc',
  'Ug8Q003_II.nc',
  'Ug16Q000_IV.nc',
  'Ug2Q010_IV.nc',
  'Ug16Q006_IV.nc'])

In [4]:
def time_average(data, timeavg):
    """
    Averages an array over specified number of time steps.

    Parameters:
    - data (numpy.ndarray): The input data array with shape (ntime, nz).
    - timeavg (int): The number of time steps over which to average.

    Returns:
    - numpy.ndarray: The averaged data array.
    """
    ntime, nz = data.shape
    # Calculate how many complete chunks we can have
    nchunks = ntime // timeavg
    # Truncate the array to make sure it can be reshaped properly
    truncated_data = data[:nchunks * timeavg]
    # Reshape the data to separate each chunk
    reshaped_data = truncated_data.reshape(nchunks, timeavg, nz)
    # Compute the mean along the new time axis (axis=1)
    averaged_data = reshaped_data.mean(axis=1)

    return averaged_data

In [5]:
time_avg = 15
paths = ['/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/']
for path in paths:
    directories, items = list_directories(path)
    for i, item in enumerate(items[:1]):
        ds_stat = Dataset( path+item )
        if 'budget' in ds_stat.groups:
            
            print ("budegt is in", item)
            sigma_th = time_average(ds_stat.groups['thermo']['th_2'][:],time_avg) # covariance of theta
            sigma_2 = time_average( ds_stat.groups['default']['w_2'][:] ,time_avg) # covarianve of w 
            Theta  = time_average( ds_stat.groups['thermo']['th'][:],time_avg) # domain mean theta
            wtheta = time_average( ds_stat.groups['thermo']['th_flux'][:],time_avg) # heat flux
            wwtheta = time_average( ds_stat.groups['budget']['wwtheta'][:],time_avg) # third moment, covarince between wtheta and w
        else:
            print (item)

/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/ ['Ug16Q006_I.nc', 'Ug8Q006_IV.nc', 'Ug8Q003_IV.nc', 'Ug16Q010_IV.nc', 'Ug2Q024_I.nc', 'Ug8Q003_III.nc', 'Ug16Q001_IV.nc', 'Ug2Q01_III.nc', 'Ug16Q003_IV.nc', 'Ug2Q010_I.nc', 'Ug1Q01_II.nc', 'Ug8Q003_II.nc', 'Ug16Q000_IV.nc', 'Ug2Q010_IV.nc', 'Ug16Q006_IV.nc']
budegt is in Ug16Q006_I.nc


## Diagnosing Constants 
Often we use conventional constant to normzalize profiles. here are some of the typically used in the PBL

In [6]:
wtheta_surface = ds_stat.groups["thermo"]['th_flux'][:,0]
pbl_height = ds_stat.groups['thermo'].variables['zi'][:]# this is a time dependent variable
grr = 9.8
T_0 = 300
wstar  = np.power( grr/T_0 * (wtheta_surface) * pbl_height , 1/3)
theta_star = wtheta_surface / wstar
scaling = wstar**2 * theta_star / pbl_height
betta = grr/T_0
ustar = ds_stat.groups['default'].variables['ustar'][:]

## From Paper: "The non-local character of turbulence asymmetry in the convective atmospheric boundary layer"

The heat-flux budget in the convective boundary layer (utilizing Boussinesq approximiation) reduces to:
$$ \frac{\partial (w \theta)}{\partial t} = 0 = \sigma^2_w \frac{d\theta}{dz} - \frac{d \overline{(w w \theta)}}{dz} - \frac{1}{p_0} \overline{\left( \theta \frac{dp}{dz} \right)} + \beta \overline{\theta^2} $$

Where, assuming the left hand side is smaller compared to terms on the right hand side: 
$$0 = -M-T-P+B$$
- $M = \sigma^2_w \frac{d\theta}{dz}$ $\rightarrow$ Mean-gradient production 
- $T = \frac{d \overline{(w w \theta)}}{dz}$ $\rightarrow$ Turbulent flux transport
- $P = \frac{1}{p_0} \overline{\left( \theta \frac{dp}{dz} \right)}$ $\rightarrow$ Pressure gradient-potential temperature covarience
- $B = \beta \overline{\theta^2}$ $\rightarrow$ buoyancy production

Additionally,
- $p_0 =$ reference state air density
- $p =$ pressure fluctuation referenced to the hydrostatic state (finite value attributed to turbulence)

In [7]:
# print short and long names of variables within each group 
### default - velocity terms
### Thermo - buoyancy or theta
### budget - TKE or heat budget terms

file_path = searchpath+'/Ug2Q010_I.nc'
df = nc.Dataset(file_path, mode='r')
for group_name, group in df.groups.items():
    print(f"Group: {group_name}")
    for var_name, var in group.variables.items():
        long_name = var.getncattr('long_name') if 'long_name' in var.ncattrs() else 'No long name'
        dimensions = var.dimensions if hasattr(var, 'dimensions') else 'No dimensions attribute'
        print(f"Variable: {var_name}, Long Name: {long_name}, Dimensions: {dimensions}")
    print("\n")
df.close()

Group: default
Variable: iter, Long Name: Iteration number, Dimensions: ('time',)
Variable: area, Long Name: Fractional area contained in mask, Dimensions: ('time', 'z')
Variable: areah, Long Name: Fractional area contained in mask, Dimensions: ('time', 'zh')
Variable: eft, Long Name: eft, Dimensions: ('time', 'z')
Variable: eft_3, Long Name: Moment 3 of the eft, Dimensions: ('time', 'z')
Variable: eft_4, Long Name: Moment 4 of the eft, Dimensions: ('time', 'z')
Variable: eft_diff, Long Name: Diffusive flux of the eft, Dimensions: ('time', 'zh')
Variable: eft_w, Long Name: Turbulent flux of the eft, Dimensions: ('time', 'zh')
Variable: eft_grad, Long Name: Gradient of the eft, Dimensions: ('time', 'zh')
Variable: eft_2, Long Name: Moment 2 of the eft, Dimensions: ('time', 'z')
Variable: eft_path, Long Name: eft path, Dimensions: ('time',)
Variable: eft_flux, Long Name: Total flux of the eft, Dimensions: ('time', 'zh')
Variable: sft, Long Name: sft, Dimensions: ('time', 'z')
Variable: s

## Compute the P term as residual
Utilizing the above equation, assuming that the left hand side is smaller compared to terms on the right hand side.

In [8]:
# calculating the P term for Ug2Q010_I.nc

df = nc.Dataset(file_path, mode='r')
default = df.groups['default']
thermo = df.groups['thermo']
budget = df.groups['budget']

#extract variables
b = thermo.variables['b'][:] #buoyancy (z)
th = thermo.variables['th'][:] #potential temperature (z)
th_2 = thermo.variables['th_2'][:] #second moment of potential temperature (z)
w = default.variables['w'][:] #vertical velocity (zh)
w_2 = default.variables['w_2'][:] #second moment of vertical velocity (zh)

#calculate varience of vertical velocity (sigma_w^2) --> for M term
w_mean = np.mean(w, axis=0)
sigma_w_sq = w_2 - (w_mean ** 2)

#check/store height levels for future interpolation
zh_levels = w.shape[1]
z_levels = th.shape[1]

In [9]:
#w and th/b have different heght dimensions, interpolate variables to w
th_interp = np.zeros_like(w)
for t in range(w.shape[0]):
    th_interp[t, :] = np.interp(np.arange(zh_levels), np.arange(z_levels), th[t, :])
th_2_interp = np.zeros_like(w)
for t in range(w.shape[0]):
    th_interp[t, :] = np.interp(np.arange(zh_levels), np.arange(z_levels), th_2[t, :])
b_interp = np.zeros_like(w)
for t in range(w.shape[0]):
    th_interp[t, :] = np.interp(np.arange(zh_levels), np.arange(z_levels), b[t, :])

#calculate product of w*th at every time step
w_theta_prod = w * th_interp
ww_theta_prod = w * w_theta_prod

#calculate mean of ww_theta_prod to get flux (/overline{wwtheta}) --> for T term
ww_theta_flux = np.mean(ww_theta_prod, axis=0)

#calculate varience of potential temperature (/overline{theta^2}) --> for B term
th_mean = np.mean(th_interp, axis=0)
th_sq = th_2_interp - th_mean ** 2

In [10]:
# calculating terms
M = sigma_w_sq * np.gradient(th_interp, axis=1) 
T = np.gradient(ww_theta_flux, axis=None) 
B = b_interp * th_sq

print("M", M.shape)
print("T",T.shape)
print("B",B.shape)

M (120, 257)
T (257,)
B (120, 257)


In [11]:
# computing P
P = -M-T+B
print(P)

[[-2.4574857500459562e-33 0.00011589168435329406 0.00020622801508267852
  ... -1.0259589543443311e-08 -1.2270136035204914e-09
  8.643319447644454e-35]
 [-2.4574857500459562e-33 0.0001161284698534734 0.0002055281426723174 ...
  -1.0131355110558015e-08 -1.213221361106684e-09 8.643319447644454e-35]
 [-2.4574857500459562e-33 0.00011670923777264405 0.00020594137183249983
  ... -1.0703017214797686e-08 -1.2829473089642863e-09
  8.643319447644454e-35]
 ...
 [-2.4574857500459562e-33 0.00011507512012385748 0.00020407200605076925
  ... -7.453111754246157e-09 -8.971583972388781e-10 8.643319447644454e-35]
 [-2.4574857500459562e-33 0.00011475179350060253 0.00020372245925767829
  ... -7.396495260923234e-09 -8.907979928358375e-10 8.643319447644454e-35]
 [-2.4574857500459562e-33 0.00011449326664193339 0.00020323849168995327
  ... -7.488175798968286e-09 -9.024246371184729e-10 8.643319447644454e-35]]


## Comparing P term and original rotta model plot
Lets reproduce plot 6 of the turbulece asymmetry paper to make sure we have all terms looking "somewhat" similar. "somewhat" because the rotta model may not be very accurate.

## Cutting out near surface part
Some of the assumptions we make when computing P as a residual may not hold near the surface. Therefore, before preparing data for EQ, we remove the near-surface part of the profile, typically the top 7-10 layers